Ushbu amaliy mashg'ulotda *Jupyter Notebook* dasturi orqali Piton dasturlash tilida Qaror daraxti muammosini o'rganamiz.

Quyidagi Piton bibliotekalarini yuklab olamiz:
- *sklearn* - Pitonda mashinaviy o'qitish algoritmlari bilan ishlash uchun *Scikit-learn* bibliotekasi. Biz ushbu bibliotekadan modelning aniqliligini o'lchashda foydalanamiz.
- *load_iris* - *sklearn* bibliotekasining iris ma'lumotlar to'plami paketini yuklab olamiz
- - *train_test_split* - *sklearn* bibliotekasining ma'lumotlar to'plamini mashq / test to'plamlarga ajratuvchi paketi
- pprint - Pitonda ma'lumotlar strukturasini o'qish uchun qulay chop etadigan biblioteka
- math - Pitonda matematik amallar bilan ishlash uchun
- *numpy* - Pitonda massivlar bilan ishlash uchun


Ushbu mashg'ulotdagi kod https://medium.com/@penggongting/implementing-decision-tree-from-scratch-in-python-c732e7c69aea ssilkadagi yechimga asoslangan.

In [32]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from pprint import pprint
import math
import numpy as np

Yuqorida ta'kidlaganimizdek, qaror daraxti muammosini o'rganish uchun *iris* ma'lumotlar to'plamidan foydalanamiz. Ushbu to'plam bilan qulayroq ishlash uchun, *sklearn* bibliotekasining *datasets* paketida *load_iris* nomli funksiyas bor va bu funksiya aynan mashinaviy o'qitish algoritmlari uchun moslangan formatdagi *iris* to'plamini taqdim etadi (ya'ni to'plam *numpy* massiv formatida, mashq va nishonlar alohida massivlarga ajratilgan va h.k.). Ushbu to'plam bilan kengroq tanishish uchun quyidagi ssilkaga o'ting: https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

Avvaliga to'plam entropiyasini hisoblaydigan quyidagi formulani Pitonda ifodalab olamiz:

$ Entropy = \sum_{i}^{n}{- p_i log_2{p_i}} $

Yuqoridagi formulada, $n$ - to'plamdagi barcha elementlar soni, $p_i$ - ma'lum bir $i$-klassning ehtimolligi bo'lib, quyidagi formula orqali ifodalasak bo'ladi:
$p_i = \frac{c_i}{n}$, bunda $c_i$ - ma'lum $i$ - klassdagi elementlar soni.

Biz yaratayotgan qaror daraxti qoida asosida to'plamni ikkiga bo'lishini hisobga olgan holda, entropiya funksiyasini to'plamda 2 ta klass mavjud deb faraz qilamiz (bunda, $c1$ - birinchi klassdagi elementlar soni, $c2$ - ikkinchi klassdagi elementlar soni)

In [33]:
def entropy_cal(c1, c2):
    """
    Returns entropy of a group of data
    c1: count of one class
    c2: count of another class
    """
    if c1== 0 or c2 == 0:  # when there is only one class in the group, entropy is 0
        return 0
    n = c1 + c2
    return -(c1*1.0/n)*math.log(c1*1.0/n, 2) -(c2*1.0/n)*math.log(c2*1.0/n, 2)

To'plam entropiya funksiyasini aniqlab oldik, ammo ushbu funksiya aniq klasslar sonini talab etadi. Amalda esa, ma'lumotlar to'plamidan biz o'zimiz klasslar sonini aniqlab olishimiz, kerak bo'ladi. Ushbu funksiyani yaratamiz, va klasslar bo'yicha har bir bo'linmaning entropiyasini yuqorida yaratilgan $entropy\_cal$ funksiyasi yordamida topamiz. Va nihoyat, bo'linmaning umumiy entropiyasini aniqlaymiz. Bo'linmaning umumiy entropiyasini quyidagicha ifodalagan edi:
$E_U = E_p - (\frac{n_{b1}}{n_U} \cdot E_{b1} + \frac{n_{b2}}{n_U} \cdot E_{b2} + ... +  \frac{n_{bn}}{n_U} \cdot E_{bn})$

Yuqoridagi formula, berilgan to'plamning entropiyasi ($ E_p $) hamda ushbu to'plam bo'linmalarining entropiyalarini ($ E_{b1}, E_{b2}, ..., E_{bn} $) hisobga olgan holda, umumiy entropiyani aniqlaydi. Lekin amaliy mashg'ulotda ushbu formulani quyidagicha aniqlaylik:

$E_U = - \frac{n_{b1}}{n_U} \cdot E_{b1} - \frac{n_{b2}}{n_U} \cdot E_{b2} - ... -  \frac{n_{bn}}{n_U} \cdot E_{bn})$

Ya'ni, bo'linuvchi to'plamning entropiyasi ($E_p$) tushirilib qoldirildi, chunki ushbu entropiyasi ($E_p$) odatda yoki unchalik ahamiyat kasb qilmaydi (agarda, ushbu bo'linuvchi to'plam qaror daraxtining ildiz elementi bo'lsa, yoki yaqin joylashgan bo'lsa), yoki undan oldingi bo'linmalarda ushbu entropiya hisobga olingan bo'ladi.

In [34]:
# get the entropy of one big circle showing above
def entropy_of_one_division(division): 
    """
    Returns entropy of a divided group of data
    Data may have multiple classes
    """
    s = 0
    n = len(division)
    classes = set(division)
    for c in classes:   # for each class, get entropy
        n_c = sum(division==c)
        e = n_c*1.0/n * entropy_cal(sum(division==c), sum(division!=c)) # weighted avg
        s += e
    return s, n

Entropiyani aniqlovchi funksiyalarni yaratib olgandan keyin, ushbu funksiyalarni qanday qilib mashq to'plamlariga tatbiq qilish mumkin degan savolga javob izlaymiz. Ushbu savolga javob izlashda, ma'lum bir qadamda qaror daraxti qoida asosida to'plamni ikkiga bo'lishini eslab o'taylik, ya'ni ushbu qoidani qanoatlantiradigan to'plam (aytaylik, $A$ to'plam) va qanoatlantirmaydigan to'plam (aytaylik, $B$ to'plam). Bizning maqsad esa, ushbu $A$ va $B$ to'plamlardagi klasslar qay darajada taqsimlanganini aniqlashdir. Albatta, bu aniqlash biz yuqorida yaratgan entropiya funksiyalari orqali aniqlanadi. Ya'ni, biz ushbu funksiyalarga bo'linma $A$ va $B$ to'plamlari uchun nishonlarni uzatishimiz kerak. Undan oldin esa, biz o'zida faqat nishon klass qiymatlarini saqlovchi $A$ va $B$ bo'linma to'plamlarining nishonlarini alohida ajratib olishimiz kerak. Bunda, biz ma'lum bir qoidaning to'plamga tatbiqini, ya'ni o'zida $True$ (qanoatlantiradigan) hamda $False$ (qanoatlantirmaydigan) qiymatlarni saqlovchi $y\_predict$ nomli massiv kerak bo'ladi. Bundan tashqari, albatta, biz ustida ishlayotgan to'plamlarning ($A$, $B$) nishonlarini o'zida saqlovchi $y\_real$ nomli massiv ham talab etiladi. $Numpy$ bibliotekasining funksionalidan foydalangan holda, $y\_real$ nishonlar massivini $y\_predict$ massivi orqali ikki to'plamga bo'lamiz. Bunda quyidagi ifodadan foydalangan holda $A$ (ya'ni ma'lum bir qoidani qanoatlantiradigan) to'plam  uchun nishon qiymatlarini olsak:

$y\_real[y\_predict]$

Quyidagi ifodadan foydalangan holda esa, $B$ (ya'ni ma'lum bir qoidani qanoatlantirmaydigan) to'plam  uchun nishon qiymatlarini olamiz:

$y\_real[\sim y\_predict]$


Yuqorida keltirilgan fikrlarni amalga oshiruvchi $get\_entropy$ funksiyasini yarataylik:

In [35]:
# The whole entropy of two big circles combined
def get_entropy(y_predict, y_real):
    """
    Returns entropy of a split
    y_predict is the split decision, True/Fasle, and y_true can be multi class
    """
    if len(y_predict) != len(y_real):
        print('They have to be the same length')
        return None
    n = len(y_real)
    s_true, n_true = entropy_of_one_division(y_real[y_predict]) # left hand side entropy
    s_false, n_false = entropy_of_one_division(y_real[~y_predict]) # right hand side entropy
    s = n_true*1.0/n * s_true + n_false*1.0/n * s_false # overall entropy, again weighted average
    return s

Entropiyaga oid kerakli barcha funksiyalarni yaratib oldik. Endi, Qaror daraxtini o'zida ifodalovchi $DecisionTreeClassifier$ Klassni yaratishga kirishamiz. Albatta ushbu klass elementlarini birma-bir yaratib olamiz, hamda so'ngida ushbu elementlardan iborat bo'lgan Qaror daraxti klassini tuzamiz.

Avvalambor, Qaror daraxtining maksimal chuqurligini belgilab beruvchi yordamchi o'zgaruvchilarni aniqlab olaylik. Eslatib o'tamiz, ushbu maksimal chuqurlikni belgilab olishdan maqsad - Qaror daraxtining $overfitting$ muammosini hamda cheksiz chuqurlashib ketishini oldini olishdir. Demak, biz $depth$ - qaror daraxtining ma'lum qadamdagi chuqurligini o'zida saqlab turuvchi, va $max\_depth$ - Qaror daraxtining maksimal chuqurligi (ya'ni qaror daraxti ushbu darajadan chuqur bo'la olmaydi) qiymatlarini o'zida saqlovchi o'zgaruvchilarni yaratib olamiz.

In [36]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth

$DecisionTreeClassifier$ klassining keyingi funksiyasi - $find\_best\_split (self, col, y)$ deb atalib, ushbu funksiya ma'lum bir ustun uchun eng minimal entropiyali to'plamlarni hosil qiluvchi qoidani topadi. Buning uchun esa, qatorlar osha yangi sikl yaratilib, shu siklning har bir qadamida ustunning mos qatoridagi qiymat bo'yicha to'plamni ikkiga bo'linadi va minimal entropiyaga solishtirib boriladi (ya'ni ushbu qator qiymati orqali bo'lingan to'plamlar entropiyasi minimal entropiyadan kichik bo'lsa, ushbu qator qiymati minimal entropiya hosil qiluvchi sifatida qabul qilinadi va h.k.). Albatta, sikl so'ngida biz eng optimal qiymatni olamiz.

In [37]:
def find_best_split(self, col, y):
    """
    col: the column we split on
    y: target var
    """
    min_entropy = 10    
    n = len(y)
    for value in set(col):  # iterating through each value in the column
        y_predict = col < value  # separate y into 2 groups
        my_entropy = get_entropy(y_predict, y)  # get entropy of this split
        if my_entropy <= min_entropy:  # check if it's the best one so far
            min_entropy = my_entropy
            cutoff = value
    return min_entropy, cutoff


Ma'lum bir ustundagi minimal entropiyani beruvchi qatorni sikl orqali topishni aniqladik, demak xuddi shu funksiyani ustunlar bo'ylab tatbiq qilib, eng effektiv ustunni (albatta, shu ustunning minimal entropiya beradigan qatorini ham) aniqlashimiz mumkin. Ushbu amalni quyidagi funksiyada aniqlaylik:

In [38]:
def find_best_split_of_all(self, x, y):
    """
    Find the best split from all features
    returns: the column to split on, the cutoff value, and the actual entropy
    """
    col = None
    min_entropy = 1
    cutoff = None
    for i, c in enumerate(x.T):  # iterating through each feature
        entropy, cur_cutoff = self.find_best_split(c, y)  # find the best split of that feature
        if entropy == 0:    # find the first perfect cutoff. Stop Iterating
            return i, cur_cutoff, entropy
        elif entropy <= min_entropy:  # check if it's best so far
            min_entropy = entropy
            col = i
            cutoff = cur_cutoff
    return col, cutoff, min_entropy

Eng yaxshi to'plamlarni aniqlaydigan barcha funksiyalarni yaratib oldik. Endi bevosita mashq jarayonini amalga oshiruvchi $fit(...)$ funksiyasini yaratishga o'tsak bo'ladi. Ushbu funksiyaning asosiy maqsadi, yuqoridagi eng yaxshi to'plamlarni aniqlaydigan funksiyalar yordamida berilgan mashq to'plami quyi to'plamlarga bo'lish orqali qoidalar Qaror daraxti shoxlariga 'ilib' chiqishdir. Ya'ni, ildiz element sifatida qaysi qoida, keyingi qadamlardagi qoidalar qanday bo'ladi va h.k.

Bundan tashqari, mashq jarayonida to'plamning qaror daraxti yordamida ishlanganini vizual tarzda ko'rish uchun topilgan qoida va so'ngi yaproq elementlarni pitonning $dictionary$ ma'lumotlar strukturasiga qayd qilib boramiz.

In [39]:
def fit(self, x, y, par_node={}, depth=0):
    """
    x: Feature set
    y: target variable
    par_node: will be the tree generated for this x and y. 
    depth: the depth of the current layer
    """
    if par_node is None:   # base case 1: tree stops at previous level
        return None
    elif len(y) == 0:   # base case 2: no data in this group
        return None
    elif self.all_same(y):   # base case 3: all y is the same in this group
        return {'val':y[0]}
    elif depth >= self.max_depth:   # base case 4: max depth reached 
        return None
    else:   # Recursively generate trees! 
        # find one split given an information gain 
        col, cutoff, entropy = self.find_best_split_of_all(x, y)   
        y_left = y[x[:, col] < cutoff]  # left hand side data
        y_right = y[x[:, col] >= cutoff]  # right hand side data
        par_node = {'col': iris.feature_names[col], 'index_col':col,
                    'cutoff':cutoff,
                   'val': np.round(np.mean(y))}  # save the information 
        # generate tree for the left hand side data
        par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)   
        # right hand side trees
        par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)  
        self.depth += 1   # increase the depth since we call fit once
        self.trees = par_node  
        return par_node
    
def all_same(self, items):
    return all(x == items[0] for x in items)

Qaror daraxtini amalga oshirish uchun deyarli barcha funksiyalar tayyor. Endi prognoz qilishni bevosita amalga oshiruvchi funksiyalarga o'tsak bo'ladi. Ushbu funksiyalar, mashq jarayonida tanlanga qoidalar asosida to'plamni bo'laklarga bo'lib prognozni amalga oshiradi.

In [40]:
    def predict(self, x):
        tree = self.trees
        results = np.array([0]*len(x))
        for i, c in enumerate(x):
            results[i] = self._get_prediction(c)
        return results
    
    def _get_prediction(self, row):
        cur_layer = self.trees
        while cur_layer.get('cutoff'):
            if row[cur_layer['index_col']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val')

Yuqorida aniqlangan funksiyalar yordamida, Qaror daraxtining Klassini tuzib chiqamiz:

In [41]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
    
    def fit(self, x, y, par_node={}, depth=0):
        if par_node is None: 
            return None
        elif len(y) == 0:
            return None
        elif self.all_same(y):
            return {'val':y[0]}
        elif depth >= self.max_depth: 
            return None
        else: 
            col, cutoff, entropy = self.find_best_split_of_all(x, y)    # find one split given an information gain 
            y_left = y[x[:, col] < cutoff]
            y_right = y[x[:, col] >= cutoff]
            par_node = {'col': iris.feature_names[col], 'index_col':col,
                        'cutoff':cutoff,
                       'val': np.round(np.mean(y))}
            par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)
            par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)
            self.depth += 1 
            self.trees = par_node
            return par_node
    
    def find_best_split_of_all(self, x, y):
        col = None
        min_entropy = 1
        cutoff = None
        for i, c in enumerate(x.T):
            entropy, cur_cutoff = self.find_best_split(c, y)
            if entropy == 0:    # find the first perfect cutoff. Stop Iterating
                return i, cur_cutoff, entropy
            elif entropy <= min_entropy:
                min_entropy = entropy
                col = i
                cutoff = cur_cutoff
        return col, cutoff, min_entropy
    
    def find_best_split(self, col, y):
        min_entropy = 10
        n = len(y)
        for value in set(col):
            y_predict = col < value
            my_entropy = get_entropy(y_predict, y)
            if my_entropy <= min_entropy:
                min_entropy = my_entropy
                cutoff = value
        return min_entropy, cutoff
    
    def all_same(self, items):
        return all(x == items[0] for x in items)
                                           
    def predict(self, x):
        tree = self.trees
        results = np.array([0]*len(x))
        for i, c in enumerate(x):
            results[i] = self._get_prediction(c)
        return results
    
    def _get_prediction(self, row):
        cur_layer = self.trees
        while cur_layer.get('cutoff'):
            if row[cur_layer['index_col']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val')

Qaror daraxti tayyor, endi ushbu qaror daraxtini $iris$ ma'lumotlar to'plamiga tatbiq qilish orqali klassifikatsiya qilamiz. Avvaliga $iris$ to'plamini yuklaylik hamda mashq va nishonlar to'plamini alohida o'zgaruvchilarga $(x, y)$ ajratib olaylik. 

In [42]:
iris = load_iris()

x = iris.data
y = iris.target

$sklearn$ bibliotekasining $train\_test\_split$ funksiyasidan foydalanib, ma'lumotlar to'plamini mashq hamda test to'plamlariga ajratib olamiz.

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

Qaror daraxtini inisializatsiya qilamiz va mashq jarayonini amalga oshiramiz, bunda mashq jarayoni qaydnomasini $mashq\_jarayoni$ o'zgaruvchisiga saqlaymiz va chop etamiz.

In [43]:
qaror_daraxti = DecisionTreeClassifier(max_depth=7)
mashq_jarayoni = qaror_daraxti.fit(x_train, y_train)
pprint (mashq_jarayoni)

{'col': 'petal width (cm)',
 'cutoff': 1.0,
 'index_col': 3,
 'left': {'val': 0},
 'right': {'col': 'petal width (cm)',
           'cutoff': 1.6,
           'index_col': 3,
           'left': {'col': 'petal length (cm)',
                    'cutoff': 5.6,
                    'index_col': 2,
                    'left': {'val': 1},
                    'right': {'val': 2},
                    'val': 1.0},
           'right': {'col': 'petal width (cm)',
                     'cutoff': 1.8,
                     'index_col': 3,
                     'left': {'col': 'petal width (cm)',
                              'cutoff': 1.7,
                              'index_col': 3,
                              'left': {'val': 2},
                              'right': {'col': 'sepal length (cm)',
                                        'cutoff': 6.7,
                                        'index_col': 0,
                                        'left': {'val': 2},
                                    

Qaror daraxtining vizual grafida ko'rsatilgan pitonning $dictionary$ qiymati - bizga mashq jarayonida qaror daraxti uchun topilgan qoidalar asosida to'plam qanday bo'linganini ko'rsatib bermoqda. E'tibor bering, bunda 

- $col$ - qoida sifatida tanlangan ustun, 
- $cutoff$ - qoida uchun tanlangan qiymat, 
- $left$ - chap bo'linma to'plam ya'ni,  $[col < cutoff]$ qoidani qanoatlantiradigan bo'linma to'plam (bunda, $value$ - prognoz qilinadigan ustun), 
- $right$ - o'ng bo'linma to'plam ya'ni, $[col < cutoff]$ qoidani qanoatlantirmaydigan bo'linma to'plam (bunda, to'plamda bir necha klasslar mavjud bo'lsa, bo'linma to'plamni yanada bo'lish amalga oshiriladi).

Endi bevosita prognoz qilish jarayoniga o'tsak.

In [44]:
predicted_y_test = qaror_daraxti.predict(x_test)
predicted_y_test

array([1, 0, 2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2,
       0, 0, 0, 2, 1, 1, 1, 1, 0, 2, 2, 0, 1, 0, 0, 1])

Endi Qaror daraxtining aniqliligini tekshiramiz. O'tgan bo'limlarda klassifikatsiya muammosi uchun o'lchovlarni ($precision$, $recall$, $f1-score$, $chalkashlik matritsasi$) yozishni o'rgangan edik, va ko'rgan edikki buning uchun bir qancha funksiyalar yozish talab etiladi. Baxtimizga $sklearn$ bibliotekasida deyarli barcha ommabop o'lchovlar uchun paket mavjud. Chalkashlik matritsasi uchun $sklearn$ bibliotekasidan $classification\_report$ funksiyasini yuklaymiz.

In [45]:
from sklearn.metrics import classification_report

$classification\_report$ funksiyasidan foydalangan holda qaror daraxti prognoz qilgan $predicted\_y\_test$ massivi hamda haqiqiy nishon qiymatlarini o'zida saqlovchi $y\_test$ o'rtasidagi chalkashlik matritsasini chop etamiz.

$F1\_score$ to'g'risida fikr  Chalkashlik matritsasini chop qilamiz.

In [49]:
print(classification_report(y_test, predicted_y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.85      0.73      0.79        15
           2       0.75      0.86      0.80        14

    accuracy                           0.84        38
   macro avg       0.87      0.86      0.86        38
weighted avg       0.85      0.84      0.84        38



Chalkashlik matritsasidan ko'rib turibmizki, 84\% nishonlar to'g'ri topilgan. $f1\_score$ o'lchovi qiymatlari quyidagicha tavsiflashimiz mumkin:
- $0$ nishonlarning barchasi to'g'ri topilgan
- $1$ nishonlarning 73\% to'g'ri topilgan
- $2$ nishonlarning 86\% to'g'ri topilgan

Umuman olib aytganda, qaror daraxtining aniqliligini ancha yaxshi deb baholashimiz mumkin.